<a href="https://colab.research.google.com/github/FibGro/Pyspark/blob/main/RDD_for_Unstructured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9ec3cf9a6ac3f63ce0246384f6de28f7b8458b02606db4ed845658eecaaf5637
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark import SparkConf, SparkContext



conf = SparkConf().setMaster("local[*]").setAppName("GHTorent")
sc = SparkContext(conf=conf)

In [4]:
rdd = sc.textFile('ghtorrent-logs.txt')

In [6]:
rdd = rdd.repartition(10)


from pyspark import StorageLevel
rdd.persist(StorageLevel.MEMORY_AND_DISK)

rdd.persist(storageLevel=StorageLevel.MEMORY_AND_DISK)

MapPartitionsRDD[10] at coalesce at NativeMethodAccessorImpl.java:0

In [7]:
# Count the number of record

print(rdd.count())

2786082


In [8]:
rdd.takeSample(False, 10)

['DEBUG, 2017-03-23T11:13:28+00:00, ghtorrent-18 -- ghtorrent.rb: User makeen-project exists',
 'DEBUG, 2017-03-23T11:19:55+00:00, ghtorrent-40 -- retriever.rb: Issue event jzse/reactjs-tictactoe 4->1005719100 exists',
 'INFO, 2017-03-24T13:51:00+00:00, ghtorrent-46 -- ghtorrent.rb: Added commit_assoc of ae556a126b0bbcbdc6d0846305c4e44f63078db6 with amateo/puppet-dovecot',
 'DEBUG, 2017-03-24T12:57:16+00:00, ghtorrent-46 -- ghtorrent.rb: Repo Tencent/mars exists',
 'DEBUG, 2017-03-23T10:06:43+00:00, ghtorrent-20 -- ghtorrent.rb: Commit gluster/gluster-kubernetes -> c0d7900ff2fba3ff604a7e6b8493eef7cbadff5e exists',
 'INFO, 2017-03-23T10:08:44+00:00, ghtorrent-18 -- retriever.rb: Added issues ShaunLim123/FYPJ -> 41',
 'DEBUG, 2017-03-23T09:19:51+00:00, ghtorrent-23 -- ghtorrent.rb: Repo ryu8179/UniRxWorkBook exists',
 'INFO, 2017-03-23T11:17:33+00:00, ghtorrent-4 -- api_client.rb: Successful request. URL: https://api.github.com/repos/mandykuo/assign-2/labels/wontfix, Remaining: 1039, Tot

In [9]:
# Get the number of lines woth both transaction or Repo information

import re
def collect_words(line):
  return re.compile('\w+').findall(line.lower())

rdd_transactions = rdd.filter(lambda line: 'transaction' in collect_words(line))
rdd_repo = rdd.filter(lambda line : 'repo' in collect_words(line))

rdd_intersect = rdd_transactions.intersection(rdd_repo)
rdd_intersect.count()

9

In [11]:
import re
def get_url(line):
  return re.findall(r'http[s]?://(?:[-\w]|(?:%[da-zA-Z]{2,}))+', line)  # Changed {2,0} to {2,}

rdd.filter(lambda line : line.split(',')[0] == 'WARN') \
    .filter(lambda line : len(get_url(line))>1) \
    .count()

12

In [ ]:
# What is the most active downloader id for Failed connections?

rdd_failed = rdd.filter(lambda line: 'failed' in collect_words(line))

# Create key and values
rdd_active_ids = rdd_failed.map(lambda line: (line.replace('--', ',').split(',')[2].split('-')[1],1))

# groupbykey
rdd_active_ids.groupByKey(4).mapValues(sum).sortBy(lambda x : x[1], ascending =False).first()

# reducebykey
rdd_active_ids.reduceByKey(lambda x,y: x+y).sortBy(lambda x : x[1], ascending =False).first()

In [ ]:
# What is the most active respository

